## Deploying a Decision Optimization model with Watson Machine Learning

This notebook shows you how to deploy a Decision Optimization model, create and monitor jobs, and get solutions using the Watson Machine Learning Python Client.


## Table of Contents
1. [Install the Watson Machine Learning client API](#setup)
2. [Create a client instance](#create)
3. [Prepare your model archive](#prepare)
4. [Upload your model on Watson Machine Learning](#upload)
5. [Create a deployment](#deploy)
6. [Create and monitor a job with inline data for your deployed model](#job)
7. [Display the solution](#display)
8. [Summary](#summary)

<a id='setup'></a>
### Set up the Watson Machine Learning client

Before you use the sample code in this notebook, you need to:

- create a <a href="https://cloud.ibm.com/catalog/services/machine-learning" target="_blank" rel="noopener noreferrer">Watson Machine Learning (WML) Service</a> instance. A free plan is offered and information about how to create the instance can be found at <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-setup.html" target="_blank" rel="noopener noreferrer"> https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/wml-setup.html.</a>


Install and then import the Watson Machine Learning client library. This notebook uses the preview Python client based on v4 of Watson Machine Learning APIs. 

**Important** Do not load both Python client libraries into a notebook.

In [1]:
# Uninstall the Watson Machine Learning client Python client based on v3 APIs

!pip uninstall watson-machine-learning-client -y

Uninstalling watson-machine-learning-client-1.0.376:
  Successfully uninstalled watson-machine-learning-client-1.0.376


In [2]:
# Install the WML client API

!pip install watson-machine-learning-client-V4

     |████████████████████████████████| 1.1MB 3.6MB/s eta 0:00:01


In [3]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

# Authenticate the Watson Machine Learning service on the IBM Cloud.

Tip: Authentication information (your credentials) can be found in the Service Credentials tab of the service instance that you created on the IBM Cloud. 
If you cannot find the instance_id field in Service Credentials, click New credential (+) to generate new authentication information.

Action: Enter your Watson Machine Learning service instance credentials here.

<a id='create'></a>
### Create a client instance

Use your Watson Machine Learning credentials.

In [4]:
# Instantiate a client using credentials
wml_credentials = {
      "apikey": "xxxxxxxxxxx",
      "instance_id": "xxxxxxxxxx",
      "url": "xxxxxxxxxx",
}

client = WatsonMachineLearningAPIClient(wml_credentials)

In [5]:
client.version

'1.0.67'

<a id='prepare'></a>
### Prepare your model archive

Put the model.py file in a subdirectory and create a tar.gz file. The model consists of two parts:
* some functions to create an `inputs` dictionary from files and create files from an `outputs` dictionary,
* the real optimization model which uses the inputs and outputs dictionaries.

Use the `write_file` command to write these models to a `main.py` file. 

Use the `tar` command to create a tar archive.

In [6]:
%mkdir model

In [7]:
%%writefile model/main.py

from docplex.util.environment import get_environment
from os.path import splitext
import pandas
from six import iteritems

def get_all_inputs():
    '''Utility method to read a list of files and return a tuple with all
    read data frames.
    Returns:
        a map { datasetname: data frame }
    '''
    result = {}
    env = get_environment()
    for iname in [f for f in os.listdir('.') if splitext(f)[1] == '.csv']:
        with env.get_input_stream(iname) as in_stream:
            df = pandas.read_csv(in_stream)
            datasetname, _ = splitext(iname)
            result[datasetname] = df
    return result

def write_all_outputs(outputs):
    '''Write all dataframes in ``outputs`` as .csv.

    Args:
        outputs: The map of outputs 'outputname' -> 'output df'
    '''
    for (name, df) in iteritems(outputs):
        csv_file = '%s.csv' % name
        print(csv_file)
        with get_environment().get_output_stream(csv_file) as fp:
            if sys.version_info[0] < 3:
                fp.write(df.to_csv(index=False, encoding='utf8'))
            else:
                fp.write(df.to_csv(index=False).encode(encoding='utf8'))
    if len(outputs) == 0:
        print("Warning: no outputs written")
        

Writing model/main.py


In [8]:
%%writefile -a model/main.py



def __iter__(self): return 0
# Load CSV files into inputs dictionnary
inputs = get_all_inputs()


import types
import pandas as pd

offers = inputs['offers']
offers = offers.loc[offers['nb_products'] >= 1]


# How much revenue is earned when selling each product
products = ["Savings", "Mortgage", "Pension"]
productValue = [200, 300, 400]
value_per_product = {products[i] : productValue[i] for i in range(len(products))}

# Total available budget
availableBudget = 25000

# For each channel, cost of making a marketing action and success factor
channels =  pd.DataFrame(data=[("gift", 20.0, 0.20), 
                               ("newsletter", 15.0, 0.05), 
                               ("seminar", 23.0, 0.30)], columns=["name", "cost", "factor"])

offersR = range(0, len(offers))
productsR = range(0, len(products))
channelsR = range(0, len(channels))


from docplex.mp.model import Model

mdl = Model("marketing_campaign")


# #### Define the decision variables
# - The integer decision variables `channelVars`, represent whether or not a customer will be made an offer for a particular product via a particular channel.
# - The integer decision variable `totaloffers` represents the total number of offers made.
# - The continuous variable `budgetSpent` represents the total cost of the offers made


channelVars = mdl.binary_var_cube(offersR, productsR, channelsR)
totaloffers = mdl.integer_var(lb=0)
budgetSpent = mdl.continuous_var()


# #### Set up the constraints
# - Offer only one product per customer.
# - Compute the budget and set a maximum for it.
# - Compute the number of offers to be made.

# In[26]:


# At most 1 product is offered to each customer
mdl.add_constraints( mdl.sum(channelVars[o,p,c] for p in productsR for c in channelsR) <=1
                   for o in offersR)

# Do not exceed the budget
mdl.add_constraint( mdl.sum(channelVars[o,p,c]*channels.at[c, "cost"]
                                           for o in offersR 
                                           for p in productsR 
                                           for c in channelsR)  <= availableBudget, "budget")  

# At least 10% offers per channel
for c in channelsR:
    mdl.add_constraint(mdl.sum(channelVars[o,p,c] for p in productsR for o in offersR) >= len(offers) // 10)

mdl.print_information()


# In[27]:


offers.reset_index(inplace=True)


# #### Express the objective
# 
# Maximize the expected revenue.

# In[28]:


obj = 0

for c in channelsR:
    for p in productsR:
        product=products[p]
        coef = channels.at[c,"factor"] * value_per_product[product]
        obj += mdl.sum(channelVars[o,p,c] * coef* offers.at[o, product] for o in offersR)

mdl.maximize(obj)


# #### Solve with Decision Optimization
# 
# Depending on the size of the problem, the solve stage might fail and require the Commercial Edition of CPLEX engines, which is included in the premium environments in Watson Studio.

# In[29]:


s = mdl.solve()
assert s, "No Solution !!!"

print(mdl.get_solve_status())
print(mdl.get_solve_details())

totaloffers = mdl.sum(channelVars[o,p,c] 
                      for o in offersR
                      for p in productsR 
                      for c in channelsR)

mdl.add_kpi(totaloffers, "nb offers")

budgetSpent = mdl.sum(channelVars[o,p,c]*channels.at[c,"cost"]
                                           for o in offersR 
                                           for p in productsR 
                                           for c in channelsR)
mdl.add_kpi(budgetSpent, "budget Spent")

for c in channelsR:
    channel = channels.at[c,"name"]
    kpi = mdl.sum(channelVars[o,p,c] for p in productsR for o in offersR)
    mdl.add_kpi(kpi, channel)
    
for p in productsR:
    product = products[p]
    kpi = mdl.sum(channelVars[o,p,c] for c in channelsR for o in offersR)
    mdl.add_kpi(kpi, product)
    
all_kpis = [(kp.name, kp.compute()) for kp in mdl.iter_kpis()]

kpis_bd = pd.DataFrame(all_kpis, columns=['kpi', 'value'])

mdl.report()

report = [(offers.at[o, "id"],channels.at[n, "name"], products[p],channels.at[c, "cost"]) 
          for c in channelsR
          for n in channelsR
          for p in productsR 
          for o in offersR  if channelVars[o,p,c].solution_value==1]

print("Marketing plan has {0} offers costing {1}".format(totaloffers.solution_value, budgetSpent.solution_value))

report_bd = pd.DataFrame(report, columns=['customer_id','channel', 'product','cost'])
    
outputs = {}
outputs['kpis'] = kpis_bd
outputs['solution'] = report_bd

# Generate output files
write_all_outputs(outputs)

Appending to model/main.py


In [9]:
import tarfile
def reset(tarinfo):
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = "root"
    return tarinfo
tar = tarfile.open("model.tar.gz", "w:gz")
tar.add("model/main.py", arcname="main.py", filter=reset)
tar.close()

<a id='upload'></a>
### Upload your model on Watson Machine Learning

Store model in Watson Machine Learning with:
* the tar archive previously created,
* metadata including the model type and runtime

Get the `model_uid`.

In [10]:
# All available runtimes

#client.runtimes.list(pre_defined=True)

In [11]:
mnist_metadata = {
    client.repository.ModelMetaNames.NAME: "MarketCampaignOpti",
    client.repository.ModelMetaNames.DESCRIPTION: "Model for Optimization",
    client.repository.ModelMetaNames.TYPE: "do-docplex_12.9",
    client.repository.ModelMetaNames.RUNTIME_UID: "do_12.9"    
}

model_details = client.repository.store_model(model='/home/dsxuser/work/model.tar.gz', meta_props=mnist_metadata)

model_uid = client.repository.get_model_uid(model_details)

print( model_uid )

952b1af4-00a2-414e-ae10-20587cc97827


<a id='deploy'></a>
### Create a deployment 

Create a batch deployment for the model, providing information such as:
* the maximum number of compute nodes
* the T-shirt size of the compute nodes

Get the `deployment_uid`.

In [12]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "MarketCampaign Optimization",
    client.deployments.ConfigurationMetaNames.DESCRIPTION: "Marketing Deployment Model",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.COMPUTE: {'name': 'S', 'nodes': 1}
}

deployment_details = client.deployments.create(model_uid, meta_props=meta_props)

deployment_uid = client.deployments.get_uid(deployment_details)

print( deployment_uid )



#######################################################################################

Synchronous deployment creation for uid: '952b1af4-00a2-414e-ae10-20587cc97827' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='7628aa36-1aae-4a1d-bd03-2c16732ec5ea'
------------------------------------------------------------------------------------------------


7628aa36-1aae-4a1d-bd03-2c16732ec5ea


In [13]:
# List all existing deployments

client.deployments.list()

------------------------------------  ---------------------------  -----  ------------------------  -------------
GUID                                  NAME                         STATE  CREATED                   ARTIFACT_TYPE
7628aa36-1aae-4a1d-bd03-2c16732ec5ea  MarketCampaign Optimization  ready  2020-04-08T16:11:42.110Z  model
985acd69-ca73-4065-9d97-2265b5e326ad  MarketCampaign Notebook      ready  2020-04-07T15:24:14.336Z  model
60275fa2-47a1-4de8-a6a1-ce15020ec4e0  WarehouseLP                  ready  2019-11-18T19:17:29.584Z  model
24bff782-6e87-4cf3-ad42-192241eacebb  Warehouse Deployment         ready  2019-11-15T14:24:18.688Z  model
4ee06bea-0341-453f-abbb-f619164ec833  CustHist                     ready  2019-10-01T01:16:00.524Z  model
b3d7292a-2eff-4fdf-9214-b14cae4135c1  PredictOffer                 ready  2019-09-30T11:38:15.369Z  model
a6ba97db-c734-46e0-ab6e-3e959ab04196  SPSSOfferPrediction          ready  2019-09-20T18:45:27.024Z  model
4041479e-947f-430e-9398-fcc986

In [14]:
#client.deployments.delete('f4788184-f92c-4dbe-9212-a07e2e016728')

### <a id='job'></a>
### Create and monitor a job with inline data for your deployed model

Create a payload containing s3 input data.

Create a new job with this payload and the deployment.

Get the `job_uid`.

In [15]:
solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA_REFERENCES: [
            {
                "id":"offers.csv",
                "type": "s3",
                "connection": {
                    "endpoint_url": "https://s3-api.us-geo.objectstorage.service.networklayer.com",
                    "access_key_id": "4d721ede2d164f319e3c94b47b55f20b",
                    "secret_access_key": "87a7709705535308917d312cdafea49ce1d86102aa86f7f7"
                },
                "location": {
                    "bucket": "marketcampaign-donotdelete-pr-jv0t7co85w5hif",
                    "path": "offers.csv"
                }   
            }
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
        {
            "id":".*\.csv"
        }
    ]
}

In [16]:
job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

print( job_uid )

7d33fb9f-9124-4730-86c4-1d0bb1c8d8e0


Display job status until it is completed.

The first job of a new deployment might take some time as a compute node must be started.

In [17]:
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print( job_details['entity']['decision_optimization']['status']['state'])

queued...
queued...
queued...
queued...
queued...
running...
running...
completed


In [18]:
print( job_details['entity']['decision_optimization']['status'])

{'state': 'completed', 'running_at': '2020-04-08T16:12:16.355Z', 'completed_at': '2020-04-08T16:12:22.042Z'}


<a id='display'></a>
### Extract and display solution

Display the output solution.

In [19]:
import pandas as pd

In [20]:
# Create a dataframe for the solution
solution = pd.DataFrame(job_details['entity']['decision_optimization']['output_data'][0]['values'], 
                        columns = job_details['entity']['decision_optimization']['output_data'][0]['fields'])
solution.head()

,kpi,value
0,nb offers,1123
1,budget Spent,24597
2,gift,112
3,newsletter,112
4,seminar,899


In [21]:
# Create a dataframe for the solution
solution_report = pd.DataFrame(job_details['entity']['decision_optimization']['output_data'][1]['values'], 
                        columns = job_details['entity']['decision_optimization']['output_data'][1]['fields'])
solution_report.head()

,customer_id,channel,product,cost
0,55728,gift,Savings,20
1,94027,gift,Savings,20
2,56258,gift,Savings,20
3,58943,gift,Savings,20
4,49240,gift,Savings,20


In [22]:
#Delete deployment model
client.deployments.delete(deployment_uid)

'SUCCESS'

<a id='summary'></a>
### Summary and next steps

You successfully completed this notebook! 

You've learned how to:

- work with the Watson Machine Learning client
- prepare your model archive and upload your model on Watson Machine Learning
- create a deployment
- create and monitor a job with inline data for your deployed model
- display the solution

Check out our online documentation at <a href="https://dataplatform.cloud.ibm.com/docs" target="_blank" rel="noopener noreferrer">https://dataplatform.cloud.ibm.com/docs</a> for more samples, tutorials and documentation. 

<hr>
Copyright © 2019. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>